## Pipeline Flavor Profile

#### 1. MS Data -> Molecules
install massbank2db ?
use sql file to build local database 

#### 2. Molecules -> OD 
Find ODs by matching molecules with databases -> Flavornet

*CAS_to_odorants.csv*

*odors_to_odorants.csv*


In [64]:
import csv

'''
get dictionary with CAS number as key and OD as value 
'''
def CAS_to_OD_Roche():
    with open('data_in_brief.tab', 'r') as tabfile:
        reader = csv.reader(tabfile, delimiter='\t')

        for row in reader:
            CAS = row[0]
            OD = row[2]
            
            print(CAS, OD)
    return

def CAS_to_molecules():
    CAS_to_molecule ={}
    with open('CAS_to_mol.txt', 'r',encoding='utf-8') as tabfile:
        reader = csv.reader(tabfile, delimiter='\t')

        for row in reader:
            CAS = row[0]
            odorant = row[2]
            odorant_single = odorant.split(' ')
            CAS_to_molecule[CAS] = odorant_single
    return CAS_to_molecule

def molecules_to_OD():
    molecule_to_OD = {}
    with open('OD_to_molecules.txt', 'r', encoding='utf-8') as file:
            reader = csv.reader(file,delimiter='\t')
            for row in reader: 
                OD = row[0]
                odorant = row[1]
                odorants = odorant.split(' ')
                for odor in odorants:
                    if odor in molecule_to_OD:
                        molecule_to_OD[odor] += [OD]
                    else:
                        molecule_to_OD[odor] = [OD]
    return molecule_to_OD

def CAS_to_OD():
    
    return

In [65]:
print(molecules_to_OD())
print(CAS_to_molecules())

{'cumin': ['acid', 'sharp'], 'aldehyde': ['acid', 'citrus', 'fat', 'flower', 'lily', 'sharp', 'spice', 'wax'], 'isovaleric': ['acid', 'rancid', 'sweat'], 'acid': ['acid', 'apple', 'balsamic', 'balsamic', 'butter', 'cheese', 'cheese', 'cheese', 'cheese', 'fat', 'fat', 'fat', 'fat', 'flower', 'green', 'honey', 'honey', 'metal', 'must', 'oil', 'pungent', 'rancid', 'rancid', 'rancid', 'rancid', 'rancid', 'sour', 'soy', 'sweat', 'sweat', 'sweat', 'sweat', 'sweat', 'sweat', 'urine'], 'propanol': ['alcohol', 'pungent'], 'pyrrolidine': ['alkaline'], 'pentane': ['alkane'], 'hexane': ['alkane'], 'octane': ['alkane'], 'nonane': ['alkane'], 'dodecane': ['alkane'], 'eicosane': ['alkane'], 'undecane': ['alkane'], 'decane': ['alkane'], 'heptane': ['alkane'], '7-heptadecene': ['alkane'], 'hexadecane': ['alkane'], 'octadecane': ['alkane'], 'tridecane': ['alkane'], 'tetradecane': ['alkane'], 'pentadecane': ['alkane'], 'heptadecane': ['alkane'], 'nonadecane': ['alkane'], 'heneicosane': ['alkane'], 'docos

##### table manipualtion done in R

```r
library(rlang)
library(tidyverse)

CAS_to_odorants <- read.table("~/Bioinformatik_20.21/Bachelorarbeit/CAS_to_mol.txt", sep = "\t", header = FALSE)
OD_to_mol <- read.table("~/Bioinformatik_20.21/Bachelorarbeit/OD_mol.txt", sep = "\t", header = FALSE)

OD_to_mol_sep <- OD_to_mol %>%
  separate_rows(V2, sep = ";\\s*") %>%
  mutate(V2 = trimws(V2))
OD_to_mol_final <- OD_to_mol_sep[, c("V2","V1")]
colnames(OD_to_mol_final) <- c("molecule", "OD")

CAS_to_odorants_sep <- CAS_to_odorants %>%
  separate_rows(V3, sep = ";\\s*") %>%
  mutate(V3 = trimws(V3))
colnames(CAS_to_odorants_sep) <- c("CAS","Mol Wt", "molecule")


merged_df <- merge(CAS_to_odorants_sep, OD_to_mol_final, by = "molecule")
write.csv(merged_df, file = "~/Bioinformatik_20.21/Bachelorarbeit/CAS_mol_OD.csv", row.names = FALSE)
```

In [85]:
mol_to_OD = {}
with open('CAS_mol_OD.csv', 'r',encoding='utf-8') as tabfile:
    reader = csv.reader(tabfile, delimiter=',')
    for row in reader: 
        molecule = row[0]
        CAS = row[1]
        Mol_wt = row[2]
        OD = row[3]
        if molecule in mol_to_OD:
            mol_to_OD[molecule].add(OD)
        else:
            mol_to_OD[molecule]  = set([OD])
        #print(molecule,CAS,Mol_wt, OD)

In [86]:
mol_to_OD

{'(-)-carvone': {'mint'},
 '(-)-cis-rose oxide': {'rose', 'sweet'},
 '(-)-cubenol': {'green tea', 'herb', 'spice'},
 '(-)-nootkatone': {'terpentine'},
 '(-)-β-bisabolene': {'balsamic'},
 '(-)-γ-elemene': {'green', 'oil', 'wood'},
 '(+)-(E)-limonene oxide': {'green'},
 '(+)-carvone': {'caraway'},
 '(+)-cis-rose oxide': {'flower', 'green'},
 '(+)-limonene,': {'citrus', 'mint'},
 '(+)-nootkatone': {'grapefruit'},
 '(+)-α-phellandrene': {'dill'},
 '(2S)-2-methyl-ethyl ester-butanoic acid': {'apple'},
 '(E)-2-dodecen-1-ol': {'oil'},
 '(E)-2-hexenal': {'green', 'leaf'},
 '(E)-2-hexenoic acid': {'fat', 'must'},
 '(E)-2-hexenol': {'green', 'leaf', 'walnut'},
 '(E)-2-nonenal': {'cucumber', 'fat', 'green'},
 '(E)-2-penten-1-ol': {'mushroom'},
 '(E)-2-undecenal': {'fat', 'green', 'soap'},
 '(E)-3-hexenol': {'fresh', 'moss'},
 '(E)-carveol': {'caraway', 'solvent'},
 '(E)-cinnamaldehyde': {'cinnamon', 'paint'},
 '(E)-dihydrocarvone': {'herb', 'warm'},
 '(E)-farnesol': {'muguet'},
 '(E)-isoelemicin'

#### 3. OD to OQ
##### Use ontology to find matching OQ

In [104]:
import pandas as pd
def OD_x_OQ():
    test = pd.read_csv('DATA_OD-2-OQ.csv', header=None)
    print(test)
    with open('DATA_OD-2-OQ.csv', 'r', encoding='utf-8') as file:
            reader = csv.reader(file,delimiter=',')
            for row in reader:
                OD = row[0]
                almond = row[1]
                woody = row[2] # usw.
    return 

OD_x_OQ()

                 0   1   2   3   4   5   6   7   8   9   ...  12  13  14  15  \
0            Acetic   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
1        Acid-acrid   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
2            Acidic   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
3             Acrid   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
4        Acrid-acid   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
..              ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..   
220            Wine   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
221      Wine-yeast   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
222           Winey   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   
223           Woody   0   1   0   0   0   0   0   0   0  ...   0   0   0   0   
224  Woody-phenolic   0   1   0   0   0   0   0   0   0  ...   0   0   0   0   

     16  17  18  19  20  21  
0     0  

#### 4. OQ to OSA
##### Use Fuzzy logic to weigh OQ with expert knowledge/Intensity from Masspec

#### 5. Show Flavourprofile